# Initialization

Импорт нужных библиотек

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [3]:
items = pd.read_parquet('goodsread/items.parquet')
events = pd.read_parquet('goodsread/events.parquet')

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [8]:
#items.head()
#events.head()

Завершите код для разбиения всех событий. В качестве точки разбиения используйте 2017-08-01, то есть отнесите в тестовую часть три последних месяца.

In [4]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx] # ваш код здесь #
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates() # ваш код здесь #
# количество пользователей, которые есть и в train, и в test
common_users = users_train[users_train.isin(users_test)] # ваш код здесь #

print(len(users_train), len(users_test), len(common_users)) 

428220 123223 120858


# === Знакомство: первые персональные рекомендации

In [5]:
cold_users =  users_test[~users_test.isin(users_train)]# ваш код здесь #
print(len(cold_users)) 

2365


Завершите код, чтобы получить топ-100 наиболее популярных книг согласно условиям выше.

In [6]:
top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()
item_popularity["popularity_weighted"] = item_popularity["users"] * item_popularity["avg_rating"]

# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values(by="popularity_weighted", ascending=False) # ваш код здесь #

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity.query('avg_rating >= 4').head(100)# ваш код здесь # 

In [7]:
top_k_pop_items.head()

,item_id,users,avg_rating,popularity_weighted
32387,18007564,20207,4.321275,87320.0
32623,18143977,19462,4.290669,83505.0
30695,16096824,16770,4.301014,72128.0
2,3,15139,4.706057,71245.0
3718,38447,14611,4.232770,61845.0


In [8]:
# добавляем информацию о книгах которые попали в топ
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 100):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_weighted", "genre_and_votes"]]) 

,item_id,author,title,publication_year,users,avg_rating,popularity_weighted,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,87320.0,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,83505.0,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,72128.0,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
3,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,71245.0,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
4,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,61845.0,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
5,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13043,4.632447,60421.0,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
6,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2012,14348,4.179189,59963.0,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
7,17927395,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016,12177,4.730640,57605.0,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
8,18692431,"Nicola Yoon, David Yoon","Everything, Everything",2015,14121,4.071454,57493.0,"{'Young Adult': 5175, 'Romance': 3234, 'Contem..."
9,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2004,11890,4.770143,56717.0,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."


Завершите предлагаемый код, чтобы в cold_users_events_with_recs для каждого события получить столбец avg_rating. В нём при совпадении по item_id будет значение из одноимённого столбца из top_k_pop_items, иначе — пропуск.
В cold_users_events_with_recs после выполнения завершённого кода должно быть столько же строк, сколько было до его выполнения.

In [10]:
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items[["item_id", "avg_rating"]], on="item_id", how="left")

cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "item_id", "rating", "avg_rating"]] 

Для какой доли событий «холодных» пользователей в events_test рекомендации в top_k_pop_items совпали по книгам? Округлите ответ до сотых.

In [11]:
# Подсчитываем количество событий "холодных" пользователей в events_test
cold_events_count = len(events_test[events_test["user_id"].isin(cold_users)])

# Подсчитываем количество событий, где top_k_pop_items совпали по книгам
matching_events_count = len(cold_users_events_with_recs.dropna(subset=["avg_rating"]))

# Вычисляем долю событий, где рекомендации совпали
matching_events_ratio = matching_events_count / cold_events_count

matching_events_ratio = round(matching_events_ratio, 2) # Округляем до сотых

print("Доля событий, где рекомендации совпали по книгам:", matching_events_ratio)

Доля событий, где рекомендации совпали по книгам: 0.2


Посчитайте метрики rmse и mae для полученных рекомендаций.

In [12]:
# посчитаем метрики рекомендаций
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])# ваш код здесь #
print(round(rmse, 2), round(mae, 2)) 

0.78 0.62


In [13]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}") 

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


Оцените степень разреженности U-I-матрицы, построенной на основе events. Используйте  данные events[['user_id', 'item_id', 'rating']]. Какое значение корректно описывает результат?

In [14]:
# Подсчет количества уникальных пар пользователь-предмет
unique_pairs_count = events[['user_id', 'item_id']].drop_duplicates().shape[0]

# Подсчет количества уникальных предметов
unique_items_count = events['item_id'].nunique()

# Вычисление степени разреженности
sparsity = 1 - (unique_pairs_count / (unique_items_count * events['user_id'].nunique()))

print("Степень разреженности U-I-матрицы: {:.2%}".format(sparsity))


Степень разреженности U-I-матрицы: 99.93%


In [15]:
# Подсчет общего количества ячеек в матрице
total_cells = events['user_id'].nunique() * events['item_id'].nunique()

# Подсчет количества пустых ячеек (ячеек без рейтинга)
empty_cells = total_cells - events.shape[0]

# Вычисление степени разреженности
sparsity = total_cells / empty_cells

print("Степень разреженности U-I-матрицы: {:.2f}".format(sparsity))

Степень разреженности U-I-матрицы: 1.00


## Реализация SVD-алгоритма

In [16]:
from surprise import Dataset, Reader
from surprise import SVD

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set)

In [17]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set) 

In [18]:
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae)

RMSE: 0.8289
MAE:  0.6474
0.8288711689059135 0.647437483750257


Проверка метрик на адекватность

In [19]:
from surprise import NormalPredictor

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set)

In [20]:
rmse = accuracy.rmse(random_predictions)
mae = accuracy.mae(random_predictions)
print(rmse, mae)

RMSE: 1.2628
MAE:  1.0018
1.2628030301013033 1.0017726877569562


Задание. Удалите из events события для редких айтемов — таких, с которыми взаимодействовало менее N пользователей. Возьмите небольшое N, например 2–3 пользователя. Получите рекомендации, посчитайте метрики, оцените, как они изменились. Подумайте, с чем могут быть связаны такие изменения.

Попробовал мануал из https://habr.com/ru/companies/otus/articles/764222/

In [21]:
# Подсчет количества уникальных пользователей, взаимодействовавших с каждым айтемом
item_users_count = events.groupby('item_id')['user_id'].nunique()

# Удаление событий для айтемов, с которыми взаимодействовали менее N пользователей
N = 2
rare_items = item_users_count[item_users_count < N].index
events_filtered = events[~events['item_id'].isin(rare_items)]

In [22]:
from surprise.model_selection import train_test_split

# Создание объекта Reader для определения структуры данных
reader = Reader(rating_scale=(1, 5))

# Преобразование DataFrame в объект Dataset формата Surprise
data = Dataset.load_from_df(events_filtered[['user_id', 'item_id', 'rating']], reader)

# Разделение данных на обучающий и тестовый наборы
trainset, testset = train_test_split(data, test_size=0.2)

In [23]:
model = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02)
# Обучение модели на обучающем наборе
model.fit(trainset)

In [24]:
# Получение прогнозов модели на тестовом наборе
predictions = model.test(testset)

# Расчет MAE и RMSE
mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

MAE:  0.6477
RMSE: 0.8307
MAE: 0.6477165525975903
RMSE: 0.8307417913595345


## Получение рекомендаций

Создайте функцию, которая позволит получить рекомендации для конкретного пользователя, используя описанный подход. Дополните прекод.

In [26]:
def get_recommendations_svd(user_id, all_items, events, model, include_seen=True, n=5):

    """ возвращает n рекомендаций для user_id """
    
    # получим список идентификаторов всех книг
    all_items = set(events['item_id'].unique())
        
    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = set(events[events['user_id'] == user_id]['item_id'].unique())
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = list(all_items - seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    predictions = [model.predict(user_id, item_id) for item_id in items_to_predict]
    
    # сортируем рекомендации по убыванию скора и берём только n первых
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["item_id", "score"])

In [27]:
get_recommendations_svd(1296647, items, events_train, svd_model)

,item_id,score
0,24812,5.000000
1,7864312,4.981188
2,25793186,4.912001
3,12174312,4.898052
4,13208,4.894869


Если нужно вывести последние события для случайного пользователя и рекомендации для него. Это можно сделать с помощью функции

In [28]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)

user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 


user_id: 1345844
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
8,Orson Scott Card,"Ender's Shadow (Ender's Shadow, #1)",2012-05-30,2012-06-16,5,"{'Science Fiction': 3299, 'Fiction': 1329, 'Yo..."
9,E.L. James,"Fifty Shades of Grey (Fifty Shades, #1)",2012-05-04,2012-06-06,3,"{'Romance': 8712, 'Fiction': 3718, 'Adult Fict..."
10,Tina Fey,Bossypants,2012-04-08,2012-05-03,2,"{'Nonfiction': 8118, 'Autobiography-Memoir': 4..."
11,"Orson Scott Card, Stefan Rudnicki, Harlan Ellison","Ender's Game (Ender's Saga, #1)",2012-04-08,2012-05-21,5,"{'Science Fiction': 15315, 'Fiction': 7488, 'Y..."
12,Kristin Kimball,"The Dirty Life: On Farming, Food, and Love",2012-03-18,2012-04-05,4,"{'Nonfiction': 420, 'Autobiography-Memoir': 35..."
13,Jonathan Franzen,Freedom,2011-10-29,2011-11-17,4,"{'Fiction': 2920, 'Contemporary': 352, 'Novels..."
14,George R.R. Martin,"A Clash of Kings (A Song of Ice and Fire, #2)",2011-07-12,2011-08-04,5,"{'Fantasy': 31452, 'Fiction': 6543, 'Fantasy-E..."
15,George R.R. Martin,"A Game of Thrones (A Song of Ice and Fire, #1)",2011-06-24,2011-07-12,4,"{'Fantasy': 44086, 'Fiction': 10111, 'Fantasy-..."
16,Ally Condie,"Matched (Matched, #1)",2011-04-28,2011-05-15,4,"{'Young Adult': 7846, 'Science Fiction-Dystopi..."
17,J.K. Rowling,Harry Potter and the Deathly Hallows (Harry Po...,2010-11-28,2010-12-26,5,"{'Fantasy': 46667, 'Young Adult': 15403, 'Fict..."


Рекомендации


,item_id,score,author,title,genre_and_votes
0,22037424,5,"J.K. Rowling, Jonny Duddle, Tomislav Tomić",Harry Potter and the Prisoner of Azkaban (Harr...,"{'Fantasy': 49994, 'Young Adult': 15433, 'Fict..."
1,24812,5,Bill Watterson,The Complete Calvin and Hobbes,"{'Sequential Art-Comics': 867, 'Humor': 378, '..."
2,24814,5,Bill Watterson,It's a Magical World: A Calvin and Hobbes Coll...,"{'Sequential Art-Comics': 680, 'Humor': 381, '..."
3,40490,5,Arthur Bennett,The Valley of Vision: A Collection of Puritan ...,"{'Christian': 141, 'Prayer': 127, 'Religion-Th..."
4,2939264,5,يحيى بن شرف النووي,رياض الصالحين,"{'Religion': 52, 'Religion-Islam': 26, 'Nonfic..."


In [29]:
my_book = items[items['author'].str.contains('Tolkien', case=False, na=False)]
print(my_book[['item_id','title']])

          item_id                                              title
57870       77678  The Lays of Beleriand (The History of Middle-E...
236592    6784686                                     Leaf by Niggle
366252      15241         The Two Towers (The Lord of the Rings, #2)
480872    1335611                                      Tree and Leaf
521574     659469                          The Hobbit: Graphic Novel
542049      16907                             Smith of Wootton Major
629237       7329       Unfinished Tales of Númenor and Middle-Earth
805720     597790                              The Children of Húrin
806150       5907                                         The Hobbit
944968   32708664                                  Beren and Lúthien
1120947      2333                                Farmer Giles of Ham
1195424      7337                                 The Tolkien Reader
1195429      7332                                   The Silmarillion
1258628     23613  Sir Gawain and 

In [30]:
events.sample()

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month
4844060,1136829,29430012,2016-12-26,2017-01-22,True,4,False,2016-12-01


In [31]:
# Добавим нового пользователя с прочитанными мной книгами
new_user_id = events['user_id'].max() + 1
#new_user_id 

books_i_like = [2187513, 1258628]  # Книги, которые мне нравятся
new_user_events = pd.DataFrame({
    'user_id': [new_user_id] * len(books_i_like),
    'item_id': books_i_like,
    'started_at': ['2012-05-30', '2012-05-30'],  # Дата начала чтения
    'read_at': ['2012-06-16', '2012-06-16'],  # Дата окончания чтения
    'is_read': [True, True],
    'rating': [5, 4],
    'is_reviewed':[False, False],
    'started_at_month':['2012-05-30', '2012-05-30']
    
})

In [32]:
new_user_events

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month
0,1430585,2187513,2012-05-30,2012-06-16,True,5,False,2012-05-30
1,1430585,1258628,2012-05-30,2012-06-16,True,4,False,2012-05-30


In [33]:
pse_events = pd.concat([events_train, new_user_events], ignore_index=True)

In [34]:
get_recommendations_svd(1430585, items, pse_events, svd_model)

,item_id,score
0,24812,5.000000
1,11221285,4.914296
2,22037424,4.908423
3,33353628,4.872179
4,54741,4.872000


In [35]:
print("Рекомендации")
user_recommendations = get_recommendations_svd(1430585, items, pse_events, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 

Рекомендации


,item_id,score,author,title,genre_and_votes
0,24812,5.000000,Bill Watterson,The Complete Calvin and Hobbes,"{'Sequential Art-Comics': 867, 'Humor': 378, '..."
1,11221285,4.914296,Brandon Sanderson,"The Way of Kings, Part 2 (The Stormlight Archi...","{'Fantasy': 641, 'Fiction': 46, 'Fantasy-Epic ..."
2,22037424,4.908423,"J.K. Rowling, Jonny Duddle, Tomislav Tomić",Harry Potter and the Prisoner of Azkaban (Harr...,"{'Fantasy': 49994, 'Young Adult': 15433, 'Fict..."
3,33353628,4.872179,Pénélope Bagieu,"Culottées #2 (Culottées, #2)","{'Sequential Art-Bande DessinÃ©e': 108, 'Femin..."
4,54741,4.872000,Quino,Toda Mafalda,"{'Sequential Art-Comics': 157, 'Humor': 47, 'S..."


# === Базовые подходы: коллаборативная фильтрация

Ниже приведён код для перекодировки идентификаторов

In [5]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])

/tmp/ipykernel_2159/1044897688.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_2159/1044897688.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_2159/1044897688.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [6]:
events_train['item_id_enc'].max()

43304

In [7]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [8]:
import sys

sum([sys.getsizeof(i) for i in user_item_matrix_train.data])/1024**3

0.26370687410235405

In [9]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train) 

/home/mle-user/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mle-user/.local/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|███████████████████████████████████████████████████████████████████████████████████| 50/50 [02:57<00:00,  3.55s/it]


Описание прочих параметров и методов класса AlternatingLeastSquares можно посмотреть на странице технической документации на библиотеку implicit.
Чтобы получить рекомендации для пользователя с помощью модели ALS, используем такую функцию:

In [41]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations

Используя get_recommendations_als, напишите код, который позволит для случайного пользователя просмотреть рекомендации в удобном формате: 
история с именами авторов и названием книг,
рекомендации с именами авторов и названием книг, seen-признаком (взаимодействовал ли уже пользователь с рекомендованной книгой).
Проанализируйте, релевантны ли рекомендации имеющейся истории.

In [42]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

user_id: 1418333


In [43]:
get_recommendations_als(user_item_matrix_train, als_model, user_id, user_encoder, item_encoder)

,item_id_enc,score,item_id
0,29392,1.294028,11235712
1,31273,1.149674,13206760
2,33783,1.127624,16096824
3,31274,1.040954,13206828
4,29910,1.026977,11870085


In [44]:
print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)

user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_als(user_item_matrix_train, als_model, user_id, user_encoder, item_encoder)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 

История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
126,"Natsuki Takaya, Alethea Nibley, Athena Nibley","Fruits Basket, Vol. 1",2012-06-30,2015-02-23,5,"{'Sequential Art-Manga': 5707, 'Fantasy': 485,..."
127,Alex Flinn,"Beastly (Beastly, #1; Kendra Chronicles, #1)",2012-01-10,2012-01-10,5,"{'Young Adult': 2672, 'Fantasy': 2570, 'Romanc..."
128,Becca Fitzpatrick,"Finale (Hush, Hush, #4)",2012-10-23,2012-10-24,5,"{'Young Adult': 1503, 'Fantasy': 1322, 'Parano..."
129,"Derek Ruiz, Becca Fitzpatrick, Jennyson Rosero...","Hush, Hush (Hush, Hush: The Graphic Novel, #1)",2014-06-02,2014-06-02,5,"{'Sequential Art-Graphic Novels': 107, 'Young ..."
130,Becca Fitzpatrick,"Silence (Hush, Hush, #3)",2012-09-21,2012-09-30,5,"{'Young Adult': 2116, 'Fantasy': 1769, 'Parano..."
131,Becca Fitzpatrick,"Hush, Hush (Hush, Hush, #1)",2012-09-07,2012-09-14,5,"{'Young Adult': 5341, 'Fantasy': 4355, 'Romanc..."
132,Becca Fitzpatrick,"Crescendo (Hush, Hush, #2)",2012-09-14,2012-09-21,5,"{'Young Adult': 2793, 'Fantasy': 2200, 'Parano..."
133,Simone Elkeles,"Chain Reaction (Perfect Chemistry, #3)",2012-07-26,2016-03-21,5,"{'Young Adult': 993, 'Romance': 993, 'Contempo..."
134,Simone Elkeles,"Rules of Attraction (Perfect Chemistry, #2)",2012-07-24,2016-03-19,5,"{'Young Adult': 1398, 'Romance': 1379, 'Contem..."
135,Simone Elkeles,"Perfect Chemistry (Perfect Chemistry, #1)",2012-07-23,2016-03-18,5,"{'Romance': 3197, 'Young Adult': 2980, 'Contem..."


Рекомендации


,item_id_enc,score,item_id,author,title,genre_and_votes
0,29392,1.294028,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)","{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
1,31273,1.149674,13206760,Marissa Meyer,"Scarlet (The Lunar Chronicles, #2)","{'Young Adult': 6930, 'Fantasy': 6267, 'Scienc..."
2,33783,1.127624,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
3,31274,1.040954,13206828,Marissa Meyer,"Cress (The Lunar Chronicles, #3)","{'Young Adult': 5813, 'Fantasy': 5413, 'Scienc..."
4,29910,1.026977,11870085,John Green,The Fault in Our Stars,"{'Young Adult': 20449, 'Fiction': 10338, 'Roma..."


Получим рекомендации для всех имеющихся пользователей. Выполните код ниже.

In [45]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, N=100)

Код возвращает рекомендации как список списков, это не очень удобно. Преобразуем его в более удобный формат — табличный.

In [46]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

In [47]:
als_recommendations.head()

,score,user_id,item_id
0,0.990941,1000000,3
1,0.896617,1000000,15881
2,0.864404,1000000,5
3,0.822254,1000000,6
4,0.774095,1000000,2


Сохраним полученные рекомендации в файл, они ещё нам пригодятся.

In [48]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("als_recommendations.parquet") 

Для удобства оценки добавим в датафрейм с рекомендациями истинные оценки из тестовой выборки:

In [49]:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "rating"]]
               .rename(columns={"rating": "rating_test"}), 
           on=["user_id", "item_id"], how="left")
) 

In [50]:
als_recommendations.sample(5)

,user_id,item_id,score,rating_test
18731091,1187310,25372801,0.228878,NaN
18557599,1185575,12578077,0.093989,NaN
6853817,1068538,16101128,0.595286,NaN
42803457,1428034,10429045,0.518817,NaN
30739204,1307392,22557272,0.101153,NaN


Подсчитать метрику NDCG для одного пользователя поможет готовая реализация из scikit-learn:

In [51]:
import sklearn.metrics

def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg

Умея считать NDCG для одного пользователя, посчитаем данную метрику, например, для 𝑘 =5 для всех пользователей из тестовой выборки. В результате каждому пользователю будет соответствовать одно значение NDCG@5. Запись “NDCG@5” означает, что метрика NDCG считается для пяти пользователей

In [52]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))

In [53]:
print(ndcg_at_5_scores.mean()) 

0.975946709792109


In [54]:
# Доля пользователей, для которых удалось посчитать метрику NDCG
sum(ndcg_at_5_scores > 0) / len(ndcg_at_5_scores)

0.35807624389737197

In [62]:
import random

# Выбор корректных индексов для нескольких айтемов
num_items_to_recommend = 10
item_ids_to_recommend = random.sample(range(len(item_encoder.classes_)), num_items_to_recommend)

# Получение рекомендаций похожих айтемов
similar_items_recommendations = als_model.similar_items(item_ids_to_recommend, N=10, filter_items=None, items=None)

# Преобразование результатов в DataFrame
similar_items_df = pd.DataFrame({
    "item_id_enc": item_ids_to_recommend,
    "similar_items": similar_items_recommendations[0].tolist(),
    "similarity_score": similar_items_recommendations[1].tolist()
})

# Вывод и анализ результатов
print(similar_items_df)

   item_id_enc                                      similar_items  \
0        32937  [32937, 36226, 37086, 10386, 40122, 36196, 150...   
1        26586  [26586, 26587, 26461, 26269, 26825, 26163, 280...   
2        25445  [25445, 25028, 38278, 32116, 19276, 19977, 209...   
3         1612  [1612, 1611, 1608, 3493, 2430, 1607, 14420, 16...   
4         9532  [9532, 2079, 17138, 12505, 29209, 17856, 14959...   
5          856  [856, 23849, 11352, 2581, 16443, 3541, 15457, ...   
6        18143  [18143, 152, 35303, 38953, 36490, 9867, 39841,...   
7        37028  [37028, 28384, 41149, 26004, 41662, 25248, 315...   
8        17168  [17168, 14990, 12169, 33353, 19958, 5715, 2581...   
9         6845  [6845, 1764, 10837, 5279, 25433, 6843, 12278, ...   

                                    similarity_score  
0  [1.0, 0.8940990567207336, 0.8940813541412354, ...  
1  [1.0000001192092896, 0.9786380529403687, 0.974...  
2  [1.0000001192092896, 0.9324819445610046, 0.911...  
3  [1.00000011920928

# === Базовые подходы: контентные рекомендации

Составим список всех возможных жанров по всем айтемам. Потом по нему будем составлять вектор с весами по жанрам для каждой книги. Из любопытства составим список жанров с подсчётом количества голосов по каждому из них.
Сначала преобразуем значения в genre_and_votes из текстового представления в тип в Python:

In [39]:
items["genre_and_votes"] = items["genre_and_votes"].apply(eval)

In [40]:
items.head(2)

,item_id,author,title,description,genre_and_votes,num_pages,average_rating,ratings_count,text_reviews_count,publisher,...,country_code,language_code,format,is_ebook,isbn,isbn13,genre_and_votes_dict,genre_and_votes_str,item_id_enc,age
3,6066819,Jennifer Weiner,Best Friends Forever,Addie Downs and Valerie Adler were eight when ...,"{'Womens Fiction-Chick Lit': 739, 'Fiction': 442}",368,3.49,51184,3282,Atria Books,...,US,eng,Hardcover,False,0743294297,9780743294294,"{'Academic': None, 'Academic-Academia': None, ...","Womens Fiction-Chick Lit 739, Fiction 442",23133,9.0
6,378460,Michael Halberstam,The Wanting of Levine,,"{'Politics': 1, 'Humor': 1}",<NA>,4.38,12,4,Berkley Publishing Group,...,US,,Paperback,False,0425040887,9780425040881,"{'Academic': None, 'Academic-Academia': None, ...","Politics 1user, Humor 1user",12687,39.0


Теперь составьте список жанров с долями голосов по ним в genres.  Дополните и выполните следующий код:

In [41]:
def get_genres(items):

    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = {}
    
    for k, v, in items.iterrows():
        genre_and_votes = v["genre_and_votes"]# ваш код здесь #
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += votes # ваш код здесь #
            except KeyError:
                genres_counter[genre] = votes

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"
    
    return genres
   
genres = get_genres(items)

In [42]:
genres

,name,votes
genre_id,,
0,Womens Fiction-Chick Lit,255297
1,Fiction,6406698
2,Politics,103297
3,Humor,304303
4,Christian,105668
...,...,...
810,German History-Nazi Party,3
811,Favorites,3
812,History-Latin American History,2


In [59]:
genres["score"] = genres["votes"] / genres["votes"].sum()
genres.sort_values(by="score", ascending=False).head(10) 

,name,votes,score
genre_id,,,
25,Fantasy,6850115,0.149498
1,Fiction,6406698,0.139821
38,Classics,3415071,0.074531
18,Young Adult,3297027,0.071955
34,Romance,2422690,0.052873
5,Nonfiction,1737798,0.037926
16,Historical-Historical Fiction,1531489,0.033423
20,Mystery,1371370,0.029929
24,Science Fiction,1218997,0.026604


Функция в коде ниже строит матрицу вида «книга-жанр». Изучите её. Подумайте, что будет соответствовать столбцам матрицы. 

In [45]:
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr

Получим матрицу с весами по жанрам для каждой книги:

In [46]:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

Дополните и выполните код ниже, чтобы получить описанную матрицу. 

In [62]:
import pandas as pd

# Заданный user_id
user_id = 1000010

# Извлечение оценок идентификаторов книг и их рейтингов для заданного user_id
user_events = events_train.query("user_id == @user_id")[["item_id", "rating"]]

# Выбор только книг, которые есть в оценках пользователя
user_items = items[items["item_id"].isin(user_events["item_id"])]

# Построение матрицы весов по жанрам для пользовательских книг
user_items_genres_csr = get_item2genre_matrix(genres, user_items)

# Вывод матрицы весов по жанрам для пользовательских книг
user_items_genres_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 149 stored elements and shape (22, 815)>

In [63]:
# Подсчет числа ненулевых элементов в user_items_genres_csr
num_nonzero_elements = user_items_genres_csr.count_nonzero()

num_nonzero_elements

149

На практике часто пользователь явно указывает предпочтения в своём профиле. У нас таких данных нет, поэтому предпочтения пользователя по жанрам вычислим автоматически на основе его истории поведени

In [64]:
# вычислим склонность пользователя к жанрам как среднее взвешенное значение популяции на его оценки книг.

# преобразуем пользовательские оценки из списка в вектор-столбец
user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)
user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0))

Можно посмотреть, какие жанры больше всего нравятся пользователю:

In [65]:
# выведем список жанров, которые предпочитает пользователь

user_genres = genres.copy()
user_genres["score"] = np.ravel(user_genres_scores)
user_genres = user_genres[user_genres["score"] > 0].sort_values(by=["score"], ascending=False)

user_genres.head(5) 

,name,votes,score
genre_id,,,
1,Fiction,6406698,0.185241
38,Classics,3415071,0.103879
25,Fantasy,6850115,0.072447
5,Nonfiction,1737798,0.050865
24,Science Fiction,1218997,0.040920


Теперь рассчитаем рекомендации на основе двух объектов:
all_items_genres_csr — матрица распределения интересов всех пользователей по жанрам. Строка — вектор распределения интересов всех пользователей по жанрам для одного объекта.
user_genres_scores — вектор интересов пользователя по жанрам.
Рекомендации будем рассчитывать с помощью косинусного сходства между двумя векторами

Чтобы посчитать косинусное сходство user_genres_scores к вектор-строкам из all_items_genres_csr, необходимо выполнить вычисления по формуле столько раз, сколько есть строк в all_items_genres_csr.

Получите наиболее релевантные рекомендации для пользователя. Дополните код так, чтобы переменная top_k_indices заполнялась индексами соответствующих книг. Для этого удобно использовать np.argsort от similarity_scores, подсчитанной для всех книг.

In [66]:
from sklearn.metrics.pairwise import cosine_similarity

# вычисляем сходство между вектором пользователя и векторами по книгам
similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

# преобразуем в одномерный массив
similarity_scores = similarity_scores.flatten()

# получаем индексы top-k (по убыванию значений), по сути, индексы книг (encoded)
k = 5
top_k_indices = np.argsort(similarity_scores)[:k]
top_k_indices

array([38862, 38838,  9716, 18969, 18925])

После вычисления top_k_indices по полученным индексам извлеките список объектов, которые могут быть интересны пользователю, при помощи кода:

In [67]:
selected_items = items[items["item_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
   display(selected_items[["author", "title", "genre_and_votes"]]) 

,author,title,genre_and_votes
1673075,Mo Willems,"Today I Will Fly! (Elephant & Piggie, #1)","{'Childrens-Picture Books': 409, 'Childrens': 199}"
1523576,Lester Sumrall,Gifts and Ministries of the Holy Spirit,"{'Christian': 6, 'Religion-Theology': 3, 'Christian-Christian Non Fiction': 2, 'Spirituality': 2}"
1422146,Nicholas Davies,Diana the Killing of a Princess,None
515584,Nayyirah Waheed,Nejma,"{'Poetry': 929, 'Feminism': 31}"
956516,"Tina Marie Kaht, Valeria Avantario",Grandparents' Day,{'Childrens': 2}


**Факультативное задание**
* Получите по алгоритму выше рекомендации для нескольких пользователей, просмотрите их на экране. Подумайте, насколько релевантны и интересны полученные рекомендации пользователям.
* Попробуйте использовать другую меру сходства для получения рекомендаций, например, евклидово расстояние. Проанализируйте, отличаются ли рекомендации от предыдущих. Подумайте почему.
* Задайте собственные предпочтения для наиболее популярных жанров. Посмотрите рекомендации для себя. Прочитали ли бы вы рекомендованные книги?

# === Базовые подходы: валидация

Посчитаем recall и precision для ALS-рекомендаций (als_recommendations). Для этого события в тестовой выборке и рекомендации для одних и тех же пользователей разметим признаками:
gt (ground truth): объект есть в тестовой выборке;
pr (predicted): объект есть в рекомендациях.
Теперь разметим признаки бинарной классификации:
TP: объект есть и в тестовой выборке, и в рекомендациях (истинная рекомендация),
FP: объекта нет в тестовой выборке, но он есть в рекомендациях (ложноположительная рекомендация),
FN: объект есть в тестовой выборке, но его нет в рекомендациях (ложноотрицательная рекомендация)

In [26]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["item_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")    

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()
    
    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common

In [69]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(
  events_train,
    events_test, 
    als_recommendations, 
    top_k=5) 

/tmp/ipykernel_4960/3166051426.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["gt"] = True


Common users: 123223


In [70]:
events_recs_for_binary_metrics.head()

,user_id,item_id,gt,score,pr,tp,fp,fn
0,1196635,18467802,True,NaN,False,False,False,True
1,1188739,10799,True,NaN,False,False,False,True
2,1001879,13206828,True,NaN,False,False,False,True
3,1001879,13206900,True,NaN,False,False,False,True
4,1001879,13206760,True,NaN,False,False,False,True


Дополните код функции compute_cls_metrics для расчёта recall. Получите значения метрик precision@5, recall@5.

In [28]:
def compute_cls_metrics(events_recs_for_binary_metric):
    
    groupper = events_recs_for_binary_metric.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum() / (groupper["tp"].sum() + groupper["fn"].sum())# ваш код здесь #
    recall = recall.fillna(0).mean()

    return precision, recall

In [72]:
precision_at_5, recall_at_5 = compute_cls_metrics(events_recs_for_binary_metrics)

rounded_recall_at_5 = round(recall_at_5, 3)
print("Recall@5:", rounded_recall_at_5)

Recall@5: 0.014


Посчитайте метрики precision@10, recall@10. Сравните их значения со значениями для precision@5, recall@10. Подумайте о причинах таких отличий.

In [73]:
als_recommendations

,user_id,item_id,score,rating_test
0,1000000,3,0.990941,NaN
1,1000000,15881,0.896617,NaN
2,1000000,5,0.864404,NaN
3,1000000,6,0.822254,NaN
4,1000000,2,0.774095,NaN
...,...,...,...,...
43058495,1430584,13206900,0.096082,NaN
43058496,1430584,5060378,0.096065,NaN
43058497,1430584,16071764,0.094949,NaN
43058498,1430584,9969571,0.094927,NaN


In [74]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(
  events_train,
    events_test, 
    als_recommendations, 
    top_k=10) 

/tmp/ipykernel_4960/3166051426.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["gt"] = True


Common users: 123223


In [75]:
precision_at_10, recall_at_10 = compute_cls_metrics(events_recs_for_binary_metrics)

rounded_precision_at_10 = round(precision_at_10, 3)
rounded_recall_at_10 = round(recall_at_10, 3)

print("Precision@10:", rounded_precision_at_10)
print("Recall@10:", rounded_recall_at_10)

Precision@10: 0.009
Recall@10: 0.031


# === Двухстадийный подход: метрики

Для рекомендаций, сохранённых в переменной als_recommendations, посчитайте покрытие по объектам согласно формуле выше. При этом используйте весь топ-100 рекомендаций.

In [11]:
als_recommendations = pd.read_parquet('als_recommendations.parquet')

In [12]:
# расчёт покрытия по объектам
cov_items = als_recommendations['item_id'].nunique() / len(items)
print(f"{cov_items:.2f}")

0.09


Посчитайте среднее Novelty@5 для als_recommendations. Для этого: разметьте каждую рекомендацию в als_recommendations булевым признаком read (False — пользователь не читал книгу, True — пользователь читал книгу), используя events_train, посчитайте Novelty@5 для каждого пользователя, посчитайте среднеарифметическое для полученных значений Novelty@5

In [13]:
events_train = events_train.rename(columns={'book_id': 'item_id'})
events_test = events_test.rename(columns={'book_id': 'item_id'})
events_train.head(1)

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month,user_id_enc,item_id_enc
0,1229132,22034,2015-07-12,2015-07-17,True,5,False,2015-07-01,229132,2460


In [14]:
# разметим каждую рекомендацию признаком read
events_train["read"] = True
als_recommendations = als_recommendations.merge(
    events_train[['read', 'user_id', 'item_id']], 
    on=["user_id", "item_id"],
    how="left"
)
als_recommendations["read"] = (
    als_recommendations["read"]
    .fillna(False)
    .astype("bool")
)

# проставим ранги
als_recommendations = als_recommendations.sort_values(
    by=['user_id', 'score'],
    ascending=[True, False]
)
als_recommendations["rank"] = (
    als_recommendations
    .groupby("user_id")
    .cumcount()
 ) + 1

# посчитаем novelty по пользователям
novelty_5 = (1-als_recommendations.query("rank <= 5").groupby("user_id")["read"].mean())

# посчитаем средний novelty
novelty_mean = novelty_5.mean()

In [15]:
novelty_mean

0.607333279143491

Используем отложенную тестовую часть данных — назовём её events_test — для получения двух новых частей данных:
одна, составляющая первые 45 дней, будет использоваться для таргетов,
другая, состоящая из 45 последних дней, будет новой тестовой выборкой.
Завершите код так, чтобы в events_labels оказалась первая часть данных, а в events_test_2 — вторая.

In [16]:
# задаём точку разбиения
split_date_for_labels = pd.to_datetime("2017-09-15").date()

split_date_for_labels_idx = events_test["started_at"] < split_date_for_labels
events_labels = events_test[split_date_for_labels_idx].copy()
events_test_2 = events_test[~split_date_for_labels_idx].copy() 
events_labels['user_id'].nunique()

99849

Объедините имеющихся кандидатов по совпадению user_id, item_id в один список.

In [17]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations = pd.read_parquet("candidates/training/als_recommendations.parquet")
content_recommendations = pd.read_parquet("candidates/training/content_recommendations.parquet")
candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=['user_id', 'item_id'],
    how="outer"
)

In [18]:
candidates.shape[0]

82993094

Дополните код ниже.
В candidates добавьте колонку target со значениями:
1 для тех item_id, которые пользователь прочитал (положительный пример).
0 — для всех остальных (негативный пример).
В candidates_for_train отберите все положительные примеры, а также не менее четырёх негативных примеров для каждого пользователя в положительных примерах.

In [19]:
# добавляем таргет к кандидатам со значением:
# — 1 для тех item_id, которые пользователь прочитал
# — 0, для всех остальных 

events_labels["target"] = 1
candidates = candidates.merge(events_labels[["user_id", "item_id", "target"]], on=['user_id', 'item_id'],
    how="left")
candidates["target"] = candidates["target"].fillna(0).astype("int")

# в кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

# для каждого пользователя оставляем только 4 негативных примера
negatives_per_user = 4
candidates_for_train = pd.concat([
    candidates_to_sample.query("target == 1"),
    candidates_to_sample.query("target == 0") \
        .groupby("user_id") \
        .apply(lambda x: x.sample(negatives_per_user, random_state=0))
    ])

In [84]:
candidates_for_train.shape[0]

213708

# === Двухстадийный подход: модель

Выше мы подготовили все нужные артефакты для обучения модели ранжирования. Выполните код, чтобы обучить модель.

In [85]:
candidates_for_train.head()

,user_id,item_id,als_score,cnt_score,target
615,1000006,29868610,0.286715,NaN,1
632,1000006,7445,0.230529,NaN,1
649,1000006,18812405,0.178382,NaN,1
1998,1000019,37415,0.043595,NaN,1
2302,1000023,7260188,0.598791,NaN,1


In [20]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6526057	total: 71.6ms	remaining: 1m 11s
100:	learn: 0.5118959	total: 1.79s	remaining: 16s
200:	learn: 0.5111710	total: 3.76s	remaining: 15s
300:	learn: 0.5105208	total: 6.64s	remaining: 15.4s
400:	learn: 0.5100174	total: 8.5s	remaining: 12.7s
500:	learn: 0.5095747	total: 10.3s	remaining: 10.3s
600:	learn: 0.5091600	total: 12.2s	remaining: 8.08s
700:	learn: 0.5087803	total: 14s	remaining: 5.99s
800:	learn: 0.5084220	total: 15.9s	remaining: 3.94s
900:	learn: 0.5080930	total: 17.7s	remaining: 1.94s
999:	learn: 0.5078081	total: 19.5s	remaining: 0us


Дополните код ниже так, чтобы в candidates_to_rank попали кандидаты от обоих базовых генераторов подобно тому, как это было сделано для фазы тренировки выше.

In [21]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations_2 = pd.read_parquet("candidates/inference/als_recommendations.parquet")
content_recommendations_2 = pd.read_parquet("candidates/inference/content_recommendations.parquet")

candidates_to_rank = pd.merge(als_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=['user_id', 'item_id'],
    how="outer")

# оставляем только тех пользователей, что есть в тестовой выборке, для экономии ресурсов
candidates_to_rank = candidates_to_rank[candidates_to_rank["user_id"].isin(events_test_2["user_id"].drop_duplicates())]
print(len(candidates_to_rank))

14517152


Дополните код для того, чтобы вызвать модель и получить для каждого пользователя топ-100 рекомендаций — значение rank нужно выставить не более ста.

In [22]:
inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

# для каждого пользователя проставляем rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id").cumcount() + 1 # ваш код здесь #

max_recommendations_per_user = 100
final_recommendations = candidates_to_rank[candidates_to_rank["rank"] <= max_recommendations_per_user]# ваш код здесь # 

In [23]:
final_recommendations.shape[0]

7519400

In [24]:
final_recommendations.to_parquet("final_recommendations_feat.parquet")

Посчитайте метрики recall и precision.
Используйте полученные рекомендации final_recommendations, отложенную тестовую выборку events_test_2, созданные в уроке «Валидация» предыдущей темы.
А также функции process_events_recs_for_binary_metrics и compute_cls_metrics.

In [29]:
events_inference = pd.concat([events_train, events_labels])

cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2,
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5)

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

Common users: 75194
precision: 0.006, recall: 0.015


# === Двухстадийный подход: построение признаков

Посчитаем новый признак — «возраст» книги на основе года публикации. Назовём его age. 
Используя обновлённый справочник объектов items, добавьте признаки возраста age и средней популярности average_rating к кандидатам для тренировки модели candidates_for_train и к кандидатам для ранжирования candidates_to_rank.

In [30]:
items["age"] = 2018-items["publication_year"]
invalid_age_idx = items["age"] < 0
items.loc[invalid_age_idx, "age"] = np.nan
items["age"] = items["age"].astype("float")

In [31]:
items.head(2)

,item_id,author,title,description,genre_and_votes,num_pages,average_rating,ratings_count,text_reviews_count,publisher,...,country_code,language_code,format,is_ebook,isbn,isbn13,genre_and_votes_dict,genre_and_votes_str,item_id_enc,age
3,6066819,Jennifer Weiner,Best Friends Forever,Addie Downs and Valerie Adler were eight when ...,"{'Womens Fiction-Chick Lit': 739, 'Fiction': 442}",368,3.49,51184,3282,Atria Books,...,US,eng,Hardcover,False,0743294297,9780743294294,"{'Academic': None, 'Academic-Academia': None, ...","Womens Fiction-Chick Lit 739, Fiction 442",23133,9.0
6,378460,Michael Halberstam,The Wanting of Levine,,"{'Politics': 1, 'Humor': 1}",<NA>,4.38,12,4,Berkley Publishing Group,...,US,,Paperback,False,0425040887,9780425040881,"{'Academic': None, 'Academic-Academia': None, ...","Politics 1user, Humor 1user",12687,39.0


In [32]:
candidates_for_train = candidates_for_train.merge(items[["item_id", "age", "average_rating"]], on="item_id", how="left")
candidates_to_rank = candidates_to_rank.merge(items[["item_id", "age", "average_rating"]], on="item_id", how="left")

In [33]:
candidates_to_rank['age'].median()

7.0

Используя события в events_train и events_inference, посчитайте и добавьте признаки пользователей к кандидатам в candidates_for_train и candidates_to_rank соответственно:
reading_years — длительность истории пользователя,
books_read — количество книг, прочитанных за всё время,
books_per_year — среднее количество прочитанных книг в год,
rating_avg — средняя оценка,
rating_std — дисперсия оценок.


In [34]:
def get_user_features(events):
    """Считает пользовательские признаки"""

    user_features = events.groupby("user_id").agg(
        reading_years=("started_at", lambda x: (x.max() - x.min()).days / 365.25),
        books_read=("item_id", "nunique"),
        rating_avg=("rating", "mean"),
        rating_std=("rating", "std"))

    user_features["books_per_year"] = user_features["books_read"] / user_features["reading_years"]

    return user_features

# Получаем пользовательские признаки для обучения
user_features_for_train = get_user_features(events_train)
candidates_for_train = candidates_for_train.merge(user_features_for_train, on="user_id", how="left")

# Оставляем только пользователей из тестовой выборки для ранжирования
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test["user_id"].drop_duplicates())]

# Получаем пользовательские признаки для ранжирования
user_features_for_ranking = get_user_features(events_inference)
candidates_to_rank = candidates_to_rank.merge(user_features_for_ranking, on="user_id", how="left")


In [35]:
candidates_for_train["books_read"].median()

32.0

Используя истории events_train и events_inference, а также ранее полученные артефакты по жанрам книг — словарь жанров genres, оценки книг по жанрам all_items_genres_csr — добавьте парные признаки, по одному на каждый жанр, которые совместно показывают, какие жанры предпочитает пользователь. 
Жанровость в данном случае — численный коэффициент принадлежности книги к жанру. Например, если пользователь прочитал три книги, которые с весами 0.3, 0.2, 0.4 из  all_items_genres_csr относятся к Fantasy, то интерес пользователя к Fantasy составляет среднее этих трёх оценок — 0.3.
Для экономии ресурсов возьмём не все жанры, а десять наиболее популярных. Все остальные отметим как не вошедшие в топ и обозначим как others. 

In [47]:
genres_top_k = 10
genres_top_idx = genres.sort_values("votes", ascending=False).head(genres_top_k).index
genres_others_idx = list(set(genres.index) - set(genres_top_idx))

genres_top_columns = [f"genre_{id}" for id in genres_top_idx]
genres_others_column = "genre_others"
genre_columns = genres_top_columns + [genres_others_column]

In [48]:
# составляем таблицу принадлежности книг к жанрам
item_genres = (
    pd.concat([
        pd.DataFrame(all_items_genres_csr[:, genres_top_idx].todense(), columns=genres_top_columns),
        pd.DataFrame(all_items_genres_csr[:, genres_others_idx].sum(axis=1), columns=[genres_others_column])
        ],
        axis=1)
    .reset_index()
    .rename(columns={"index": "item_id_enc"})
)

In [49]:
# объединяем информацию принадлежности книг к жанрам с основной информацией о книгах
items = items.merge(item_genres, on="item_id_enc", how="left")

def get_user_genres(events, items, item_genre_columns):
    user_genres = (
        events
        .merge(items[["item_id"] + item_genre_columns], on="item_id", how="left")
        .groupby("user_id")[item_genre_columns].mean()
    )
    return user_genres

In [50]:
user_genres_for_train = get_user_genres(events_train, items, genre_columns)# ваш код здесь #
candidates_for_train = candidates_for_train.merge(user_genres_for_train, on="user_id", how="left")

user_genres_for_ranking = get_user_genres(events_inference, items, genre_columns)
candidates_to_rank = candidates_to_rank.merge(user_genres_for_ranking, on="user_id", how="left") 

Какой получилось медиана жанровости книг в candidates_for_train для жанра “Romance”? Ответ округлите до сотых.

In [51]:
candidates_for_train.head()

,user_id,item_id,als_score,cnt_score,target,age,average_rating,reading_years,books_read,rating_avg,...,genre_1,genre_38,genre_18,genre_34,genre_5,genre_16,genre_20,genre_24,genre_33,genre_others
0,1000006,29868610,0.286715,NaN,1,NaN,3.90,1.820671,17.0,4.294118,...,0.246138,0.105182,0.057684,0.010375,0.078927,0.004294,0.021665,0.008603,0.0,0.286282
1,1000006,7445,0.230529,NaN,1,12.0,4.24,1.820671,17.0,4.294118,...,0.246138,0.105182,0.057684,0.010375,0.078927,0.004294,0.021665,0.008603,0.0,0.286282
2,1000006,18812405,0.178382,NaN,1,4.0,3.81,1.820671,17.0,4.294118,...,0.246138,0.105182,0.057684,0.010375,0.078927,0.004294,0.021665,0.008603,0.0,0.286282
3,1000019,37415,0.043595,NaN,1,12.0,3.87,0.276523,6.0,4.166667,...,0.158224,0.000000,0.000000,0.000000,0.195082,0.000000,0.000000,0.082617,0.0,0.514445
4,1000023,7260188,0.598791,NaN,1,8.0,4.03,0.005476,2.0,3.500000,...,0.170366,0.000000,0.159612,0.019622,0.000000,0.000000,0.000000,0.077326,0.0,0.254950


In [52]:
genres.loc[genres['name'] == 'Romance']

,name,votes
genre_id,,
34,Romance,2422690


In [53]:
display(candidates_for_train['genre_34'].median().round(2))

0.04

Обучите модель, выполнив код ниже:

In [54]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score', 
    'age', 'average_rating', 'reading_years', 'books_read', 
    'rating_avg', 'rating_std', 
    'books_per_year'] + genre_columns
target = 'target'

# создаём Pool
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0,
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6485043	total: 26.2ms	remaining: 26.1s
100:	learn: 0.4665268	total: 2.58s	remaining: 23s
200:	learn: 0.4578610	total: 5.09s	remaining: 20.2s
300:	learn: 0.4518428	total: 7.63s	remaining: 17.7s
400:	learn: 0.4471687	total: 10.1s	remaining: 15.2s
500:	learn: 0.4429055	total: 13.2s	remaining: 13.2s
600:	learn: 0.4390647	total: 15.8s	remaining: 10.5s
700:	learn: 0.4355651	total: 18.3s	remaining: 7.79s
800:	learn: 0.4321506	total: 20.8s	remaining: 5.17s
900:	learn: 0.4288350	total: 23.4s	remaining: 2.57s
999:	learn: 0.4257325	total: 25.9s	remaining: 0us


Получите топ-100 самых релевантных рекомендация для каждого пользователя, используя обученную модель.

In [55]:
inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1] # ваш код здесь #

# для каждого пользователя проставим rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] =   candidates_to_rank.groupby("user_id").cumcount() + 1 # ваш код здесь #

max_recommendations_per_user = 100
final_recommendations = candidates_to_rank.query("rank <= @max_recommendations_per_user")

In [56]:
final_recommendations.head()

,user_id,item_id,als_score,cnt_score,cb_score,rank,age,average_rating,reading_years,books_read,...,genre_1,genre_38,genre_18,genre_34,genre_5,genre_16,genre_20,genre_24,genre_33,genre_others
1,1000003,7260188,1.129979,NaN,0.969223,1,8.0,4.03,7.4141,94.0,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
3,1000003,2767052,1.112699,NaN,0.962315,2,10.0,4.34,7.4141,94.0,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
2,1000003,6148028,1.123475,NaN,0.951618,3,9.0,4.30,7.4141,94.0,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
98,1000003,9361589,1.060634,NaN,0.574411,4,7.0,4.03,7.4141,94.0,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
31,1000003,38447,0.537152,NaN,0.563894,5,20.0,4.07,7.4141,94.0,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763


In [57]:
final_recommendations['user_id'].nunique()

75194

Используя отложенную тестовую выборку events_test_2, посчитайте метрики recall и precision для полученных рекомендаций.

In [58]:
# для экономии ресурсов оставим события только тех пользователей, 
# для которых следует оценить рекомендации
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test_2["user_id"].drop_duplicates())]

In [59]:
cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2,
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5)

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

Common users: 75194
precision: 0.011, recall: 0.030


Любопытно понять, какие признаки вносят наибольший вклад в ранжирование. Алгоритм CatBoost позволяет получить такую информацию (англ. feature importance), которая генерируется во время тренировки модели. Для этого используйте метод get_feature_importance(). 

Выполните код для получения информации о важности признаков. Выведите список признаков feature_importance в порядке убывания их важности.

In [60]:
feature_importance = pd.DataFrame(cb_model.get_feature_importance(), 
    index=features, 
    columns=["fi"])
feature_importance = feature_importance.sort_values(by='fi', ascending=False) # ваш код здесь #

print(feature_importance ) 

                       fi
als_score       30.043771
age             18.356837
average_rating  13.889881
books_read       7.140439
cnt_score        2.887655
reading_years    2.599439
genre_18         2.558679
genre_others     2.522204
genre_1          2.519999
genre_25         2.444803
genre_34         2.211371
books_per_year   1.740982
genre_38         1.588057
genre_20         1.582231
rating_avg       1.570182
genre_33         1.507410
genre_16         1.425483
genre_24         1.334677
genre_5          1.077762
rating_std       0.998138


# Шаг 1. Набор похожих объектов

Чтобы получить набор похожих объектов, можно воспользоваться уже известным алгоритмом ALS из библиотеки implicit, у которого на такой случай есть удобный метод similar_items (подробнее о нём вы можете прочитать в официальной документации).
Воспользуемся им и получим по 10 самых похожих айтемов.
Задание 1 из 6
Дополните код ниже, чтобы получить набор похожих объектов в similar_items. Вы можете подглядеть решение в уроке «Коллаборативная фильтрация: ALS» — там вы реализовывали похожую логику для получения персональных рекомендаций.

In [66]:
train_item_ids_enc = events_train['item_id_enc'].unique()
max_similar_items = 10

In [68]:
# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items+1)

In [69]:
# преобразуем полученные списки в табличный формат
sim_item_item_ids_enc = similar_items[0]
sim_item_scores = similar_items[1]

In [70]:
similar_items = pd.DataFrame({
    "item_id_enc": train_item_ids_enc,
    "sim_item_id_enc": sim_item_item_ids_enc.tolist(), 
    "score": sim_item_scores.tolist()})

In [102]:
similar_items.head()

,score,item_id_1,item_id_2
11,1.000000,22034,22026
12,0.922490,22034,22026
13,0.874765,22034,22026
14,0.873763,22034,22026
15,0.850654,22034,22026


In [72]:
# разворачиваю списки в датафрейме
similar_items = similar_items.explode("sim_item_id_enc", ignore_index=True)
similar_items = similar_items.explode("score", ignore_index=True)

In [73]:
similar_items.head(3)

,item_id_enc,sim_item_id_enc,score
0,2460,2460,1.0
1,2460,2460,0.92249
2,2460,2460,0.874765


In [74]:
# приводим типы данных
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype("int")# ваш код здесь #
similar_items["score"] = similar_items["score"].astype("float")

In [75]:
similar_items.sample(5)

,item_id_enc,sim_item_id_enc,score
4174446,24179,12072,0.740413
3916677,10300,14505,0.686747
2424515,11432,10802,0.864366
419400,7495,4225,0.934908
4650209,12627,16883,0.619270


In [76]:
# получаем изначальные идентификаторы
similar_items["item_id_1"] = item_encoder.inverse_transform(similar_items["item_id_enc"])# ваш код здесь #
similar_items["item_id_2"] = item_encoder.inverse_transform(similar_items["sim_item_id_enc"])# ваш код здесь #
similar_items = similar_items.drop(columns=["item_id_enc", "sim_item_id_enc"])

# убираем пары с одинаковыми объектами
similar_items = similar_items.query("item_id_1 != item_id_2")

In [84]:
similar_items.sample(5)

,score,item_id_1,item_id_2
3447692,0.907505,19545859,17570751
3068336,0.918075,1970527,40923
3265575,0.898279,9499320,9658159
4924353,0.733572,197132,22883285
2082422,0.820152,168760,1020951


In [85]:
similar_items.to_parquet("similar_items.parquet") 

In [103]:
def print_sim_items(item_id, similar_items):

    item_columns_to_use = ["item_id", "author", "title", "genre_and_votes", "average_rating", "ratings_count"]
    
    item_id_1 = items.query("item_id == @item_id")[item_columns_to_use]
    display(item_id_1)
    
    si = similar_items.query("item_id_1 == @item_id")
    si = si.merge(items[item_columns_to_use].set_index("item_id"), left_on="item_id_2", right_index=True)
    display(si) 

In [105]:
print_sim_items(16299, similar_items) 

,item_id,author,title,genre_and_votes,average_rating,ratings_count
1991,16299,Agatha Christie,And Then There Were None,"{'Mystery': 12703, 'Classics': 6623, 'Fiction'...",4.23,429352


,score,item_id_1,item_id_2,author,title,genre_and_votes,average_rating,ratings_count
26268,1.000000,16299,16328,Agatha Christie,"The Murder of Roger Ackroyd (Hercule Poirot, #4)","{'Mystery': 5069, 'Fiction': 1765, 'Classics':...",4.20,74002
26269,0.858777,16299,16328,Agatha Christie,"The Murder of Roger Ackroyd (Hercule Poirot, #4)","{'Mystery': 5069, 'Fiction': 1765, 'Classics':...",4.20,74002
26270,0.817708,16299,16328,Agatha Christie,"The Murder of Roger Ackroyd (Hercule Poirot, #4)","{'Mystery': 5069, 'Fiction': 1765, 'Classics':...",4.20,74002
26271,0.811799,16299,16328,Agatha Christie,"The Murder of Roger Ackroyd (Hercule Poirot, #4)","{'Mystery': 5069, 'Fiction': 1765, 'Classics':...",4.20,74002
26272,0.794610,16299,16328,Agatha Christie,"The Murder of Roger Ackroyd (Hercule Poirot, #4)","{'Mystery': 5069, 'Fiction': 1765, 'Classics':...",4.20,74002
...,...,...,...,...,...,...,...,...
26373,0.781295,16299,16366,Agatha Christie,Endless Night,"{'Mystery': 1032, 'Fiction': 289, 'Mystery-Cri...",3.75,10155
26374,0.776483,16299,16366,Agatha Christie,Endless Night,"{'Mystery': 1032, 'Fiction': 289, 'Mystery-Cri...",3.75,10155
26375,0.774083,16299,16366,Agatha Christie,Endless Night,"{'Mystery': 1032, 'Fiction': 289, 'Mystery-Cri...",3.75,10155
26376,0.769652,16299,16366,Agatha Christie,Endless Night,"{'Mystery': 1032, 'Fiction': 289, 'Mystery-Cri...",3.75,10155


In [90]:
print_sim_items(17245, similar_items)

,item_id,author,title,genre_and_votes,average_rating,ratings_count
2119,17245,"Bram Stoker, Nina Auerbach, David J. Skal",Dracula,"{'Classics': 19603, 'Horror': 10601, 'Fiction'...",3.98,636895


,score,item_id_1,item_id_2,author,title,genre_and_votes,average_rating,ratings_count
263307,1.000000,17245,480204,"Gaston Leroux, Alexander Teixeira de Mattos",The Phantom of the Opera,"{'Classics': 7010, 'Fiction': 2103, 'Horror': ...",3.97,144859
263308,0.928823,17245,480204,"Gaston Leroux, Alexander Teixeira de Mattos",The Phantom of the Opera,"{'Classics': 7010, 'Fiction': 2103, 'Horror': ...",3.97,144859
263309,0.900337,17245,480204,"Gaston Leroux, Alexander Teixeira de Mattos",The Phantom of the Opera,"{'Classics': 7010, 'Fiction': 2103, 'Horror': ...",3.97,144859
263310,0.898938,17245,480204,"Gaston Leroux, Alexander Teixeira de Mattos",The Phantom of the Opera,"{'Classics': 7010, 'Fiction': 2103, 'Horror': ...",3.97,144859
263311,0.897706,17245,480204,"Gaston Leroux, Alexander Teixeira de Mattos",The Phantom of the Opera,"{'Classics': 7010, 'Fiction': 2103, 'Horror': ...",3.97,144859
...,...,...,...,...,...,...,...,...
263412,0.895993,17245,1953,"Charles Dickens, Richard Maxwell",A Tale of Two Cities,"{'Classics': 20021, 'Fiction': 6969, 'Historic...",3.82,646983
263413,0.886899,17245,1953,"Charles Dickens, Richard Maxwell",A Tale of Two Cities,"{'Classics': 20021, 'Fiction': 6969, 'Historic...",3.82,646983
263414,0.881911,17245,1953,"Charles Dickens, Richard Maxwell",A Tale of Two Cities,"{'Classics': 20021, 'Fiction': 6969, 'Historic...",3.82,646983
263415,0.878392,17245,1953,"Charles Dickens, Richard Maxwell",A Tale of Two Cities,"{'Classics': 20021, 'Fiction': 6969, 'Historic...",3.82,646983


In [2]:
print_sim_items(17245, similar_items)

NameError: name 'print_sim_items' is not defined

In [ ]:
import logging

from fastapi import FastAPI
from contextlib import asynccontextmanager

logger = logging.getLogger("uvicorn.error")

app = FastAPI()

@asynccontextmanager
async def lifespan(app: FastAPI):
    # код ниже (до yield) выполнится только один раз при запуске сервиса
    logger.info("Starting")
    yield
    # этот код выполнится только один раз при остановке сервиса
    logger.info("Stopping")
import logging as logger
import pandas as pd

class Recommendations:

    def __init__(self):

        self._recs = {"personal": None, "default": None}
        self._stats = {
            "request_personal_count": 0,
            "request_default_count": 0,
        }

    def load(self, type, path, **kwargs):
        """
        Загружает рекомендации из файла
        """

        logger.info(f"Loading recommendations, type: {type}")
        self._recs[type] = pd.read_parquet(path, **kwargs)
        if type == "personal":
            self._recs[type] = self._recs[type].set_index("user_id")
        logger.info(f"Loaded")

    def get(self, user_id: int, k: int=100):
        """
        Возвращает список рекомендаций для пользователя
        """
        try:
            recs = self._recs["personal"].loc[user_id]
            recs = recs["item_id"].to_list()[:k]
            self._stats["request_personal_count"] += 1
        except KeyError:
            recs = self._recs["default"]
            recs = recs["item_id"].to_list()[:k]
            self._stats["request_default_count"] += 1
        except:
            logger.error("No recommendations found")
            recs = []

        return recs

    def stats(self):

        logger.info("Stats for recommendations")
        for name, value in self._stats.items():
            logger.info(f"{name:<30} {value} ")    
# создаём приложение FastAPI
app = FastAPI(title="recommendations", lifespan=lifespan)

rec_store = Recommendations()

rec_store.load(
    "personal",
    "/home/mle-user/mle_projects/mle-recsys-start/final_recommendations_feat.parquet",
    columns=["user_id", "item_id", "rank"],
)
rec_store.load(
    "default",
    "/home/mle-user/mle_projects/mle-recsys-start/top_recs.parquet",
    columns=["item_id", "rank"],
)


@app.post("/recommendations")
async def recommendations(user_id: int, k: int = 100):
    """
    Возвращает список рекомендаций длиной k для пользователя user_id
    """

    recs = rec_store.get(user_id, k)

    return {"recs": recs}

In [91]:
import requests
recommendations_url = 'http://127.0.0.1:8000/recommendations' # ваш код здесь #
features_store_url = "http://127.0.0.1:8010"
events_store_url = "http://127.0.0.1:8020"

In [98]:
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 1291248, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs) 

{'detail': 'Not Found'}


In [101]:
params = {"user_id": 1291248, "item_id": 17245}

resp = requests.post(events_store_url + "/put", headers=headers, params=params)

In [100]:
import requests

events_store_url = "http://127.0.0.1:8020"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 1337055}

resp = requests.post(events_store_url + "/get", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")
    
print(result)

{'user_id': 1337055, 'events': []}


In [ ]:
events_store_url = "http://127.0.0.1:8020"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 1127794, "item_id": 4731479}

resp = requests.post(events_store_url + "/put", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")